In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Time-variant Properties (e.g. coupon schedule) in LUSID 

Attributes
----------
coupon schedules
multi-valued properties
time-variant properties
"""

toggle_code("Toggle Docstring")

## Multi-value Properties

This notebook illustrates the use of multi-value properties, which are a type of property that can store a set of values, such as a schedule or other sequence defined by the user. You can learn more about types of LUSID properties [here](#https://support.finbourne.com/what-is-a-property), this particular sample notebook focuses on a property with constraint-style `Collection`. 

In the example below we use a quarterly ratings schedule as a demonstrative example, showing how the LUSID API can be used to query a schedule of key value pairs, and then parse for the effective dates and values.

In [2]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import libraries
from datetime import datetime, timedelta
import time
import pytz
import json
import os
import re
import pandas as pd

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Configure notebook logging and warnings
import logging
logging.basicConfig(level=logging.INFO)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

# Import required LUSID APIs
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.6.5682.0


## 1. Load Data

### 1.1 Portfolio

Load a portfolio with the scope of this notebook. Currently a multi-value property can be assigned to a portfolio of instruments, so below we will define an empty placeholder and upsert it to LUSID using a dataframe. 

In [3]:
scope = "MultiValueSchedule"
code = "RatingSchedule"

# Setup a dataframe from which we will creat the portfolio
data = {'portfolio_code':  [code],
        'portfolio_name': [code],
       }

portfolio_df= pd.DataFrame(data, columns=['portfolio_code','portfolio_name'])

# Create a mapping schema for the portfolio
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [4]:
# A portfolio can be loaded using a dataframe with file_type = "portfolios"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=portfolio_df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])

,success,failed
0,1,0


## 2. Multi-Value Property

### 2.1 Setup property definition

With the instruments in LUSID, we can now define the properties by calling the `PropertyDefinitionsApi`, where we will begin by setting our [**property_definition**](https://support.finbourne.com/what-is-a-property-definition) using the required constraint-style for a label-value set. 

In [5]:
# Setup property definition
property_scope = "MultiValue"
property_code = "QuarterlyRating"

def create_property(property_scope, property_code):
    # Create the property definition request
    property_definition = models.CreatePropertyDefinitionRequest(
                domain="Portfolio",
                scope=property_scope,
                code=property_code,
                display_name=f"MV-{code}",
                # Set the constraint style to 'Collection'
                constraint_style="Collection",
                data_type_id=lusid.ResourceId(scope="system", code="string"),
            )

            # create property definition
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=property_definition
        )
    except lusid.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            logging.info(
                f"Property {property_definition.domain}/{property_definition.scope}/{property_definition.code} already exists"
            )
    return property_definition

property_definition = create_property(property_scope, property_code)

INFO:root:Property Portfolio/MultiValue/QuarterlyRating already exists


With the property defined, we can use the `PortfoliosApi` to upsert the label-value-set to our desired portfolio which will be matched by the portfolio's scope and code.

In [6]:
# Create property key from definition
property_key = f"{property_definition.domain}/{property_definition.scope}/{property_definition.code}"

# Add properties to portfolio
def upsert_schedule(portfolio_scope, portfolio_code, property_key, schedule):
    portfolios_api.upsert_portfolio_properties(
        scope=portfolio_scope,
        code=portfolio_code,
        request_body={
            property_key : models.ModelProperty(
                key=property_key,
                value=models.PropertyValue(
                    label_value_set=models.LabelValueSet(values=schedule)
                ),
            )
        },
    )

schedule = [
    '{ "2019-12-31" : "5"}',
    '{ "2020-03-31" : "4"}',
    '{ "2020-06-30" : "3"}',
    '{ "2020-09-30" : "3"}',
]

# Use the portfolio identifiers and property key to upsert the schedule
upsert_schedule(scope, code, property_key, schedule)

### 2.2 Query properties from LUSID

With the properties now in LUSID, we can use the the [**get_portfolio_properties**](https://www.lusid.com/docs/api#operation/GetPortfolioProperties) call to the API in order to see the label value set.

In [7]:
# Build a function to query the property label-value-set
def get_portfolio_schedule(portfolio_code, portfolio_scope, property_key):
    portfolio_properties = portfolios_api.get_portfolio_properties(
            scope=portfolio_scope, code=portfolio_code
        ).properties
        
    return portfolio_properties[property_key].value.label_value_set.values

# Pull the schedule and save as a new variable
requested_schedule = get_portfolio_schedule(code, scope, property_key)

# We can store the schedule requested from LUSID in a dictionary
dates = {}
for pair in schedule:
    date = (re.findall(r'"(.*?)"', pair))
    dates[datetime.strptime(date[0], "%Y-%m-%d").astimezone(pytz.utc)] = date[1]

We can now query the schedule using an effective date, for a forward-looking view of the upcoming ratings.

In [8]:
def get_upcoming_schedule(effectiveAt):
    upcoming_schedule={}
    for key, value in dates.items():
        if key >= datetime.strptime(effectiveAt, "%Y-%m-%d").astimezone(pytz.utc):
            upcoming_schedule[key] = value
    df = pd.DataFrame.from_dict(upcoming_schedule, orient = "index", columns = ['Values'])
    return df.sort_index()

get_upcoming_schedule("2020-03-29")

,Values
2020-03-30 23:00:00+00:00,4
2020-06-29 23:00:00+00:00,3
2020-09-29 23:00:00+00:00,3
